# HuggingFace Crash Course
Pipelines, Pretrained Model, Fine Tuning  
Inspired by this [YouTube tutorial](https://youtu.be/GSt00_-0ncQ) (using pytorch).  
Prerequisite:  `pip install transformers`

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification # for pytorch, same model but remove 'TF' in front
import tensorflow as tf
import torch
import torch.nn.functional as F

## [Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)  
An easy way to use models for inference, e.g. sentiment analysis, image classification, object detection, question answering

In [ ]:
classifier = pipeline('sentiment-analysis')

# feeding a sample text through pipeline
result = classifier('We are very happy to show you the HuggingFace Transformers Library.')
print(result)

In [ ]:
# feeding a list of text through pipeline
X_train = ["We are very happy to show you the HuggingFace Transformers Library.", "We hope you don't hate it."]
results = classifier(X_train)

for result in results:
    print(result)

### Using a specific model in pipeline
[AutoModels documentation](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html)  
- for pytorch, `import AutoModelForSequenceClassification from transformers`
- for tensorflow, `import TFAutoModelForSequenceClassification from transformers`  

[AutoTokenizer documentation](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#autotokenizer)

In [ ]:
model_name = 'bert-base-uncased'

model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

classifier1 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

## Using Model and Tokenizer without pipeline
Note: you'll get same results with or without pipeline!  
Recommended to use pipeline unless you want to finetune model

In [ ]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english' # 'bert-base-uncased'

model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

token_ids = tokenizer("We are very happy to show you the HuggingFace Transformers Library.")
print(token_ids)
print('In input_ids, tokens 101 and 102 are the beginning and end of string tokens')

See [PreTrainedTokenizer documentation](https://huggingface.co/transformers/v3.0.2/main_classes/tokenizer.html#transformers.PreTrainedTokenizer) for parameter details.  
Specifically for `return_tensors` parameter: click [here](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__.return_tensors)  

In [ ]:
X_train = ["We are very happy to show you the HuggingFace Transformers Library.", "We hope you don't hate it."]

batch = tokenizer(X_train, padding=True, truncation=True, max_length=512, return_tensors='tf')
print(batch)

In [ ]:
outputs = model(batch)
predictions = tf.nn.softmax(outputs.logits, axis=-1)
labels = tf.argmax(predictions, axis=-1)
labels = [model.config.id2label[label_id] for label_id in labels.numpy()] # convert from label_id to class name

print(outputs)
print(predictions)
print(labels)

## Saving and Reloading Saved Models and Tokenizers

In [ ]:
# save model and tokenizer (esp. after finetuning)
save_directory = 'saved'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = TFAutoModelForSequenceClassification.from_pretrained(save_directory)

## [Model Hub](https://huggingface.co/models)
Use any model uploaded by community  
To use any model, copy the name in your chosen model (e.g. `oliverguhr/german-sentiment-bert`)

In [ ]:
model_name = 'oliverguhr/german-sentiment-bert'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

X_train_german = ["Mit keinem guten Ergebnis", "Das war unfair", # negative
            "nicht so schlecht wie erwartet", "Das war gut!", # positive
            "Sie fahrt ein grunes Auto"] # neutral

batch = tokenizer(X_train_german, padding=True, truncation=True, max_length=512, return_tensors='tf')
outputs = model(batch)
predictions = tf.nn.softmax(outputs.logits, axis=-1)
labels = tf.argmax(predictions, axis=-1)
labels = [model.config.id2label[label_id] for label_id in labels.numpy()]

print(labels)
print('Correct answer should be:')
print('[negative, negative, positive, positive, neutral]')

## Challenge: Implement the model J2s used for TIL NLP
[`harshit345/xlsr-wav2vec-speech-emotion-recognition`](https://huggingface.co/harshit345/xlsr-wav2vec-speech-emotion-recognition)

In [ ]:
# requirement packages
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2FeatureExtractor
import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "harshit345/xlsr-wav2vec-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech
def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    inputs = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to(device) for key in inputs}
    with torch.no_grad():
        logits = model(**inputs).logits
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

In [ ]:
# path for a sample
path = ''   
outputs = predict(path, sampling_rate)